In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pbn_37k
from pbn_37k.ask import sigAssistant

import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from dotenv import load_dotenv
load_dotenv()

brain = sigAssistant(pathCache=".cache/", llm_highend="gpt-4o",  llm_fast="gpt-4o-mini")

/home/kelu/projets/holbaek/.venv/lib/python3.11/site-packages/langchain/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


In [3]:
from langchain_community.chat_models import ChatPerplexity
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache
from langchain_perplexity import ChatPerplexity


import pandas as pd
set_llm_cache(SQLiteCache(database_path=".cache/.langchain.db"))



In [4]:
bkgchat = ChatPerplexity(temperature=0.1, model="sonar-pro")
#  ?https://docs.perplexity.ai/api-reference/chat-completions-post#body-search-after-date-filter
def perplex(prompt, size="low"):
    response = bkgchat.invoke(prompt, extra_body={"web_search_options": {"search_context_size": size}} )
    x = response.model_dump()["content"]
    citations = response.model_dump()['additional_kwargs']["citations"]
    for n in range(len(citations)):
        x = x.replace("["+str(n+1)+"]","[["+citations[n]+"]]")

    #pair = json.loads(x)
    return x

In [5]:
with open("data/stickynotes.md","r") as f:
    STICKIES = f.read()
    stickynotes = list(set([x.strip() for x in STICKIES.split("\n\n") if len(x.strip()) ]))
stickynotes

['City densifying',
 'RES-plants',
 'Less burocracy in civil society',
 'Stay as long as possible at home',
 'Jyderup city center',
 'Planning for architectural history and green standards',
 'Business strategy',
 'Preservation of architectural and cultural environment',
 'Holbeak Sports by(?)',
 'Citizen focus',
 'Better indoor climate in schools, kindergarten',
 'DUT - roads from school to leisure',
 'Building DGNB CO2 reduction',
 '2050 CO2 neutral',
 'Energy Islands',
 'Reetlex grid support, electricity use',
 'More citizens',
 'Social supply chain, work conditions',
 'Social funds project to get youth into jobs or education',
 'Biodiversity',
 'Lighting strategy focus on safety',
 'Attract more education opportunities to Holbeak',
 'Digitalisation in procedures, processes',
 'Better community schools',
 'Preservation of landscape',
 'Tram',
 'Tollose street zone',
 'Better infrasturccture, "fjordstien"',
 'Car fleet transormation to EV',
 'Getting young into education or work',
 '

In [6]:
context = "I have been running a workshop for city planners about how to make Holbaek a better place for all."\
            "What can you tell me in particular about the '$STICKY$' topic that is specific to the context of Holbaek," \
            " with online information about the topic. **IMPORTANT** If you don't find anything relevant to the topic, just tell me 'THERES NOTHING SPECIFIC' only."


In [ ]:
stickies = []
for topic in stickynotes:
    #print(topic)
    a =     perplex(context.replace("$STICKY$",topic), size="high")
    stickies.append( [topic, a] )
stickies = pd.DataFrame(stickies, columns=["topic","content"])
stickies["source"] = "Hoelbaek"
stickies.to_csv("data/stickynotes_with_online_context.csv", index=False)
stickies

,topic,content
0,City densifying,THERES NOTHING SPECIFIC
1,RES-plants,THERES NOTHING SPECIFIC
2,Less burocracy in civil society,THERES NOTHING SPECIFIC
3,Stay as long as possible at home,THERES NOTHING SPECIFIC
4,Jyderup city center,THERES NOTHING SPECIFIC
5,Planning for architectural history and green s...,"Based on the search results, there is **highly..."
6,Business strategy,THERES NOTHING SPECIFIC
7,Preservation of architectural and cultural env...,"Based on the search results, Holbæk has severa..."
8,Holbeak Sports by(?),Holbæk Sportsby is a significant sports comple...
9,Citizen focus,**THERES NOTHING SPECIFIC**


In [ ]:
REDO = stickies[stickies.content.str.contains("THERES NOTHING SPECIFIC")].topic.unique()

redoes = []
for topic in stickynotes:
    #print(topic)
    a =     perplex(context.replace("$STICKY$",topic).replace("Holbaek","Denmark"), size="high")
    redoes.append( [topic, a] )
redoes = pd.DataFrame(redoes, columns=["topic","content"])
redoes["source"] = "Denmark"
redoes

In [9]:
Qs = ["What are the particularities of Holbæk (denmark) that make it a unique municipality?",
"Give me an overview of the history of Holbæk (denmark) from its founding to the present day.",
"Describe the cultural and recreational activities available in Holbæk (denmark) for residents and visitors.",
"How has the economy of Holbæk (denmark) evolved over the years, and what are the main industries today?",
"Discuss the geographical features of Holbæk (denmark) and how they influence the lifestyle of its inhabitants.",
"Who are some notable figures from Holbæk (denmark) and what contributions have they made to their fields?",
"What are the educational institutions like in Holbæk (denmark), and how do they serve the local community?",
"Explore the transportation infrastructure in Holbæk (denmark) and its connectivity to other regions.",
"What environmental initiatives are in place in Holbæk (denmark) to promote sustainability and conservation?"
    ]
QA = []
for q in Qs:
    if len(q):
        a = perplex(q, size="high")
        QA.append([q,a])

In [10]:
HEADER = """---
layout: default
title: TITLE
parent: PARENT
has_children: true
nav_order: NAVORDER
---\n\n"""

In [11]:
CITIES = {}
VISIONS = []
ACTIVITIES = []

for i in range(len(cities)):
    desc = cities[i]
    q = prompt_template.replace("$CITYNAME$", desc)
    urban_report = brain.ask(q)
    for k in range(3):
        urban_report = urban_report.replace("Page "+str(k+1)+": ", "").replace("---","")
    

    analysis = brain.analyseText( txt=urban_report,
            TypeOfItem = "Vision",
            Source="maptivity.online",
            Place=cn[i].title(),
            MIN=15,
            Reviewed=True,
            MODEL="gpt-4o-mini")
        
    CITIES[cn[i]] = {"report": urban_report}
    VISIONS.append(urban_report)

    report = HEADER.replace("TITLE", cn[i].title()).replace("NAVORDER", str(i+3)).replace("\nparent: PARENT","") + "![](places/"+cn[i]+".png)\n\n" + urban_report
    report = report.replace("## **Overview & Sustainability Profile**","## Overview & Sustainability Profile")
    SCORE = "\n\n### ISO37101 mapping for '"+analysis.Source_Title.unique()[0]+"'\n\n#### Scores\n\n" + analysis[["Score","Purpose","Issue","Justification"]].to_markdown(index=False)+"\n\n"
    report = report + SCORE
    
    with open(f"docs/{cn[i]}_report.md", "w") as f:
        f.write(report)

NameError: name 'cities' is not defined

# Reviews

In [ ]:
%pip freeze > requirements.txt